In [1]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,tensorflow,keras

Wed Feb 27 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn not installed
pandas 0.23.4
tensorflow 1.12.0
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Using TensorFlow backend.


In [2]:
# TODO visualise and summarise the corpus data

In [3]:
from data.preprocess import PreProcessor

data = PreProcessor(config=PreProcessor.DEFAULT_CONFIG,
                    data_dir='data/raw/r252-corpus-features/org/elasticsearch/action/admin/cluster/allocation/')

vocab = data.metadata['token_vocab']
processed = data.get_tensorise_data()


In [24]:
from tensorflow.python import keras
from tensorflow.python.keras import layers
import numpy as np
from models.cnn_attention import ConvAttention

embedding_dim = 128
vocabulary_size = len(vocab)
max_chunk_length = data.config['max_chunk_length']
code_snippet = processed['body_tokens']
label_name = np.expand_dims(processed['name_tokens'], 2)
# label_name = keras.utils.to_categorical(processed['name_tokens'], num_classes=vocabulary_size)
print("Vocab Size: {} Code snippet len: {} label_name len: {}".format(vocabulary_size, len(code_snippet), len(label_name)))

# TODO make the input a json file and parse it
batch_size = 1
k1 = 8
k2 = 8
w1 = 24
w2 = 29
w3 = 10
dropout_rate = 0.5

# Optimised hyperparameter are reported in page 5 of the paper

# define layers
main_input = layers.Input(shape=(max_chunk_length,),
                          batch_size=batch_size,
                          dtype='int32', name='main_input')

cnn_layer = ConvAttention(vocabulary_size=vocabulary_size,
                          embedding_dim=embedding_dim,
                          max_chunk_length=max_chunk_length,
                          k1=k1,
                          k2=k2,
                          w1=w1,
                          w2=w2,
                          w3=w3,
                          dropout_rate=dropout_rate)

optimizer = keras.optimizers.Nadam()  # RMSprop with Nesterov momentum
loss_func = keras.losses.mean_squared_error

# define execution
cnn_output = cnn_layer(main_input)
model = keras.Model(inputs=[main_input], outputs=cnn_output)
model.compile(optimizer=optimizer,
              loss=loss_func,
              metrics=['accuracy'])
# fit the model
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

history = model.fit(code_snippet, label_name, epochs=5, verbose=2, batch_size=batch_size, callbacks=[tbCallBack])


Vocab Size: 319 Code snippet len: 135 label_name len: 135
ConvAttention: Tokens shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: C shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: L_1 shape = (1, 50, 8)


AttentionFeatures: L_2 shape = (1, 50, 8)
AttentionFeatures: L_2 shape  after multiply = (1, 50, 8)
AttentionFeatures: L_feat shape = Tensor("conv_attention_1/attention_features_1/l2_normalize:0", shape=(1, 50, 8), dtype=float32)
ConvAttention: L_feat shape = (1, 50, 8)
AttentionWeights: l_feat shape = (1, 50, 8)
AttentionWeights: attention_weight shape = (1, 50, 1)
ConvAttention: n_hat shape = (1, 128)
ConvAttention: n shape = (128, 50, 1)


/home/samialab/anaconda3/envs/method-name-prediction/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5


 - 5s - loss: 1607.3305 - acc: 0.0013


...

 - 3436s - loss: 1607.3305 - acc: 0.0013


Epoch 4/5


 - 4s - loss: 1607.3305 - acc: 0.0013


Epoch 5/5


 - 4s - loss: 1607.3305 - acc: 0.0013


In [47]:
layer_name = 'conv_attention'
model.predict(code_snippet[0].reshape(1, -1), steps=1).shape

# model.predict(code_snippet[0].reshape(1, -1), batch_size=1)

(128, 50, 1)

In [9]:
model.summary()
# # overfit and evaluate the model 
loss, accuracy = model.evaluate(code_snippet, label_name, batch_size=1, verbose=0)
print('Accuracy: {}'.format(accuracy * 100))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (1, 50)                   0         
_________________________________________________________________
conv_attention (ConvAttentio (128, 50, 1)              70968     
Total params: 70,968
Trainable params: 70,968
Non-trainable params: 0
_________________________________________________________________


Accuracy: 0.1333333303531011


In [16]:
# translate prediction

from data.utils import translate_tokenized_array_to_list_words

# prediction = model.predict(code_snippet[5].reshape(1, -1))
# translate_tokenized_array_to_list_words(vocab, processed['body_tokens'][1])
# print(prediction)
# vocab.get_name_for_id(317)
# processed['body_tokens'][0]

keras.utils.to_categorical(processed['name_tokens'], num_classes=vocabulary_size).shape

(135, 50, 319)

In [32]:
code_snippet[0].reshape(1, -1).shape

(1, 50)